In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import warnings
import nltk
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('hindi-train.csv', encoding="utf-8")

In [3]:
def count_length():
    df['word_count'] = df['headline'].apply(lambda x: len(str(x).split(" ")))

In [4]:
count_length()

In [5]:
df.head(10)

,headline,Label,word_count
0,"मेलबर्न एयरपोर्ट पर घंटों फंसे नोवाक जोकोविच, ...",Sports,19
1,अर्जेंटीना के स्टार फुटबॉलर लियोनल मेसी कोविड-...,Sports,17
2,"SAI का बड़ा फैसला, कैंपों और प्रैक्टिस सेंटर म...",Sports,18
3,Pro Kabaddi League: ऑलराउंडर रोहित गूलिया के द...,Sports,20
4,टेनिस: टाटा ओपन महाराष्ट्र में भारत के युकी भा...,Sports,13
5,Pro Kabaddi League: पटना पायरेट्स और बंगाल वॉर...,Sports,17
6,"I League: भारतीय फुटबॉल में कोरोना विस्फोट, AI...",Sports,17
7,'दो चैंपियन' के साथ ट्रेनिंग करते नजर आए शिखर ...,Sports,19
8,IND vs SA: जोहानिसबर्ग टेस्ट में पुजारा और रहा...,Sports,21
9,कप्तानी विवाद पर पूर्व पाक कप्तान सलमान बट ने ...,Sports,19


In [6]:
stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्ह', 'तनह', 'तस', 'तस', 'त', 'थ', 'थ', 'थ', 'दबर', 'दय', 'दसर', 'दसर', 'द', 'दवर', 'न', 'नह', 'न', 'नहयत', 'नच', 'न', 'पर', 'पर', 'पहल', 'पर', 'प', 'फर', 'बन', 'बह', 'बहत', 'बद', 'बल', 'बलकल', 'भ', 'भतर', 'मगर', 'मन', 'म', 'म', 'यद', 'यह', 'यह', 'यह', 'य', 'यह', 'य', 'रख', 'रह', 'रह', 'ऱवस', 'लए', 'लय', 'लकन', 'व', 'वरग', 'वह', 'वह', 'वह', 'वह', 'वल', 'वह', 'व', 'वग़रह', 'सग', 'सकत', 'सकत', 'सबस', 'सभ', 'सथ', 'सबत', 'सभ', 'सर', 'स', 'स', 'ह', 'हआ', 'हई', 'हए', 'ह', 'ह', 'ह', 'हत', 'हत', 'हत', 'हन', 'हन', 'अपन', 'जस', 'हत', 'सभ', 'तह', 'इह', 'दवर', 'इस', 'कह', 'थ', 'उह', 'ओर', 'जह', 'वह', 'अभ', 'बन', 'ह', 'उह', 'उह', 'ह', 'वगरह', 'एस', 'रवस', 'कन', 'नच', 'कफ', 'उस', 'पर', 'भतर', 'ह', 'बह', 'वह', 'कइ', 'यह', 'जह', 'तह', 'कस', 'कइ', 'यह', 'इह', 'जधर', 'इह', 'अद', 'इतयद', 'हइ', 'कनस', 'इसक', 'दसर', 'जह', 'अप', 'कह', 'उनक', 'भ', 'वरग', 'हअ', 'जस', 'नह']
stopwords_en = stopwords.words('english')
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
stopwords_lst = stopwords_hi + stopwords_en + punctuations

In [7]:
df['label_id'] = df.Label.factorize()[0]

def processText(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text 

In [8]:
for i in range(len(df)):
    df['headline'][i] = processText(df['headline'][i])
count_length()

In [9]:
df.head(10)

,headline,Label,word_count,label_id
0,"मेलबर्न एयरपोर्ट पर घंटों फंसे नोवाक जोकोविच, ...",Sports,19,0
1,अर्जेंटीना के स्टार फुटबॉलर लियोनल मेसी कोविड-...,Sports,17,0
2,"का बड़ा फैसला, कैंपों और प्रैक्टिस सेंटर में ...",Sports,18,0
3,: ऑलराउंडर रोहित गूलिया के दम पर हरियाणा स्ट...,Sports,20,0
4,टेनिस: टाटा ओपन महाराष्ट्र में भारत के युकी भा...,Sports,13,0
5,: पटना पायरेट्स और बंगाल वॉरियर्स को मिली जी...,Sports,17,0
6,": भारतीय फुटबॉल में कोरोना विस्फोट, ने सप्त...",Sports,17,0
7,'दो चैंपियन' के साथ ट्रेनिंग करते नजर आए शिखर ...,Sports,19,0
8,: जोहानिसबर्ग टेस्ट में पुजारा और रहाणे रहे ...,Sports,21,0
9,कप्तानी विवाद पर पूर्व पाक कप्तान सलमान बट ने ...,Sports,19,0


In [10]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens

In [11]:
df['headline'] = df.headline.apply(lambda x: tokenization(x))
df.head(10)

,headline,Label,word_count,label_id
0,"[मेलबर्न, एयरपोर्ट, पर, घंटों, फंसे, नोवाक, जो...",Sports,19,0
1,"[अर्जेंटीना, के, स्टार, फुटबॉलर, लियोनल, मेसी,...",Sports,17,0
2,"[का, बड़ा, फैसला, ,, कैंपों, और, प्रैक्टिस, से...",Sports,18,0
3,"[:, ऑलराउंडर, रोहित, गूलिया, के, दम, पर, हरिया...",Sports,20,0
4,"[टेनिस, :, टाटा, ओपन, महाराष्ट्र, में, भारत, क...",Sports,13,0
5,"[:, पटना, पायरेट्स, और, बंगाल, वॉरियर्स, को, म...",Sports,17,0
6,"[:, भारतीय, फुटबॉल, में, कोरोना, विस्फोट, ,, न...",Sports,17,0
7,"[', दो, चैंपियन, ', के, साथ, ट्रेनिंग, करते, न...",Sports,19,0
8,"[:, जोहानिसबर्ग, टेस्ट, में, पुजारा, और, रहाणे...",Sports,21,0
9,"[कप्तानी, विवाद, पर, पूर्व, पाक, कप्तान, सलमान...",Sports,19,0


In [12]:
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (stopwords_lst)]
count_length()
df.head(10)

,headline,Label,word_count,label_id
0,"[मेलबर्न, एयरपोर्ट, घंटों, फंसे, नोवाक, जोकोवि...",Sports,16,0
1,"[अर्जेंटीना, स्टार, फुटबॉलर, लियोनल, मेसी, कोव...",Sports,14,0
2,"[बड़ा, फैसला, कैंपों, प्रैक्टिस, सेंटर, में, प...",Sports,13,0
3,"[ऑलराउंडर, रोहित, गूलिया, दम, हरियाणा, स्टीलर्...",Sports,13,0
4,"[टेनिस, टाटा, ओपन, महाराष्ट्र, में, भारत, युकी...",Sports,10,0
5,"[पटना, पायरेट्स, बंगाल, वॉरियर्स, मिली, जीत, ट...",Sports,9,0
6,"[भारतीय, फुटबॉल, में, कोरोना, विस्फोट, ने, सप्...",Sports,10,0
7,"[दो, चैंपियन, साथ, ट्रेनिंग, नजर, आए, शिखर, धव...",Sports,15,0
8,"[जोहानिसबर्ग, टेस्ट, में, पुजारा, रहाणे, रहे, ...",Sports,15,0
9,"[कप्तानी, विवाद, पूर्व, पाक, कप्तान, सलमान, बट...",Sports,15,0


In [13]:
df['headline'] = df['headline'].str.join(" ")
df.head(10)

,headline,Label,word_count,label_id
0,मेलबर्न एयरपोर्ट घंटों फंसे नोवाक जोकोविच फिर ...,Sports,16,0
1,अर्जेंटीना स्टार फुटबॉलर लियोनल मेसी कोविड टेस...,Sports,14,0
2,बड़ा फैसला कैंपों प्रैक्टिस सेंटर में पहुंचने ...,Sports,13,0
3,ऑलराउंडर रोहित गूलिया दम हरियाणा स्टीलर्स ने य...,Sports,13,0
4,टेनिस टाटा ओपन महाराष्ट्र में भारत युकी भांबरी...,Sports,10,0
5,पटना पायरेट्स बंगाल वॉरियर्स मिली जीत टाइटंस त...,Sports,9,0
6,भारतीय फुटबॉल में कोरोना विस्फोट ने सप्ताह लिए...,Sports,10,0
7,दो चैंपियन साथ ट्रेनिंग नजर आए शिखर धवन रोहित ...,Sports,15,0
8,जोहानिसबर्ग टेस्ट में पुजारा रहाणे रहे फ्लॉप ट...,Sports,15,0
9,कप्तानी विवाद पूर्व पाक कप्तान सलमान बट ने राह...,Sports,15,0


In [14]:
# set parameters:
max_features = 20000
maxlen = 400   
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10

In [15]:
X = df['headline'].values
y = df['label_id']
y = to_categorical(y, num_classes=5)
y.shape

(37741, 5)

In [16]:
print('Loading data...')
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test  = tokenizer.texts_to_sequences(x_test)

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test  =  pad_sequences(x_test, padding='post', maxlen=maxlen)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test  =  pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
30192 train sequences
7549 test sequences
Pad sequences (samples x time)
x_train shape: (30192, 400)
x_test shape: (7549, 400)


In [17]:
test_classes = []
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        if y_test[i][j] == 1:
            if j==0:
                test_classes.append(0)
            if j==1:
                test_classes.append(1)
            if j==2:
                test_classes.append(2)
            if j==3:
                test_classes.append(3)
            if j==4:
                test_classes.append(4)

In [18]:
import json
tokenizer_json = tokenizer.to_json()
with open('hi_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=True))

In [19]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(5))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 50)           1000000   
                                                                 
 dropout (Dropout)           (None, 400, 50)           0         
                                                                 
 conv1d (Conv1D)             (None, 398, 250)          37750     
                                                                 
 global_max_pooling1d (Globa  (None, 250)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 250)               62750     
                                                                 
 dropout_1 (Dropout)         (None, 250)               0         
                                         

In [20]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/10
944/944 [==============================] - 89s 94ms/step - loss: 0.4756 - accuracy: 0.8046 - val_loss: 0.2725 - val_accuracy: 0.8785
Epoch 2/10
944/944 [==============================] - 79s 83ms/step - loss: 0.2107 - accuracy: 0.9007 - val_loss: 0.2715 - val_accuracy: 0.8791
Epoch 3/10
944/944 [==============================] - 80s 85ms/step - loss: 0.1755 - accuracy: 0.9089 - val_loss: 0.2852 - val_accuracy: 0.8793
Epoch 4/10
944/944 [==============================] - 90s 95ms/step - loss: 0.1635 - accuracy: 0.9124 - val_loss: 0.2979 - val_accuracy: 0.8764
Epoch 5/10
944/944 [==============================] - 98s 103ms/step - loss: 0.1617 - accuracy: 0.9127 - val_loss: 0.3410 - val_accuracy: 0.8777
Epoch 6/10
944/944 [==============================] - 92s 98ms/step - loss: 0.1585 - accuracy: 0.9122 - val_loss: 0.3982 - val_accuracy: 0.8681
Epoch 7/10
944/944 [==============================] - 92s 97ms/step - loss: 0.1586 - accuracy: 0.9131 - val_loss: 0.3421 - val_accuracy

In [21]:
loss, Accuracy = model.evaluate(x_test, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", Accuracy)

236/236 [==============================] - 5s 20ms/step - loss: 0.3794 - accuracy: 0.8767
Test Loss: 0.37940824031829834
Test Accuracy: 0.8766723871231079


In [22]:
model.save('HI_CNN.h5')
model.save_weights('HI_CNN_weights.h5')

In [22]:
predicted_categories = []
preds = model.predict(x_test)
for i in preds:
    pred_classes = np.argsort(i)[-1:][::-1]
    predicted_categories.append(pred_classes[0])

In [23]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
report = classification_report(test_classes, predicted_categories)
print(report)

              precision    recall  f1-score   support

           0       0.98      0.80      0.88      1510
           1       0.70      0.90      0.78      1516
           2       0.94      0.96      0.95      1470
           3       0.96      0.96      0.96      1543
           4       0.87      0.76      0.81      1510

    accuracy                           0.88      7549
   macro avg       0.89      0.88      0.88      7549
weighted avg       0.89      0.88      0.88      7549



In [24]:
mat = confusion_matrix(test_classes, predicted_categories)
mat

array([[1210,  235,    9,   27,   29],
       [   4, 1360,   35,    2,  115],
       [   6,   30, 1413,   13,    8],
       [   9,    0,   33, 1484,   17],
       [  10,  328,    7,   13, 1152]], dtype=int64)

In [25]:
import json
from keras_preprocessing.text import tokenizer_from_json

def load():
    model = load_model('HI_CNN.h5')
    maxlen = 400
    id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

    with open('hi_tokenizer.json') as f:
        data = json.load(f)
        tokenizer = tokenizer_from_json(data)
load()

In [26]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input1 =["सेंसेक्स 1129 पॉइंट्स लुढ़का, रिलायंस का शेयर 4.5% टूटा, मार्केट कैप 2 घंटे में 7.88 लाख करोड़ घटा"]

input_sequences = tokenizer.texts_to_sequences(my_input1)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result1 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result1[c] = round(p*100,2)
result1

{'business': 100.0,
 'entertainment': 0.35,
 'tech': 0.05,
 'politics': 0.05,
 'sports': 0.0}

In [27]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input2 =["शादी की खबरों के बीच कटरीना कैफ पर भड़के विक्की कौशल, दोनों के बीच हुआ झगड़ा! जानें वजह"]

input_sequences = tokenizer.texts_to_sequences(my_input2)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result2 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result2[c] = round(p*100,2)
result2

{'entertainment': 100.0,
 'sports': 0.07,
 'tech': 0.02,
 'business': 0.0,
 'politics': 0.0}

In [28]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input3 =["पश्चिमी यूपी से पूर्वांचल, अवध से बुंदेलखंड तक, कैसे कमल की फसल को खाद दे रहे पीएम?"]

input_sequences = tokenizer.texts_to_sequences(my_input3)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result3 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result3[c] = round(p*100,2)
result3

{'politics': 99.99,
 'entertainment': 28.42,
 'sports': 1.21,
 'business': 0.67,
 'tech': 0.01}

In [29]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input4 =["भारत पहुंची पाकिस्तान की टीम, 24 नवंबर से भुवनेश्वर में खेला जाएगा वर्ल्ड कपाा"]

input_sequences = tokenizer.texts_to_sequences(my_input4)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result4 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result4[c] = round(p*100,2)
result4

{'sports': 99.55,
 'entertainment': 25.63,
 'tech': 6.27,
 'politics': 0.94,
 'business': 0.08}

In [30]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input5 =["सावधान! अगर ये 7 ऐप्स आपके फोन में हों तो तुरंत करें डिलीट, बन सकते हैं खतरा"]

input_sequences = tokenizer.texts_to_sequences(my_input5)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result5 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result5[c] = round(p*100,2)
result5

{'tech': 99.99,
 'business': 28.4,
 'politics': 0.01,
 'sports': 0.01,
 'entertainment': 0.0}